# Whoo hoo statcast

Could it get better? Physics data available for me to play with!

In [1]:
import pandas as pd 
import numpy as np


In [2]:
statcast = pd.read_csv('../data/hitters_statcast_since2016.csv').dropna()
statcast['Barrel%'] = statcast['Barrel%'].apply(lambda x: float(x.strip('%')))
statcast['HardHit%'] = statcast['HardHit%'].apply(lambda x: float(x.strip('%')))

##Introduce a number of physical quantities, very approximate.
statcast['cosLA'] = np.cos(statcast['LA']/180*np.pi)
statcast['sinLA'] = np.sin(statcast['LA']/180*np.pi)
statcast['EV_x'] = statcast['EV']*statcast['cosLA']
statcast['EV_y'] = statcast['EV']*statcast['sinLA']
acceleration = -32 * 60 * 60 / 5280
func = (lambda x: -2 * x / acceleration)
statcast['Hangtime'] = func(statcast['EV_y'])
statcast['Distance'] = statcast['Hangtime']*statcast['EV_x']

## We know that the optimal LA for homeruns is not 0 degrees.
## Below, we found that an average LA of 20 was very good for HR
## hitting. We score this by the cosine squared.

statcast['LA_optimality'] = np.cos((statcast['LA'] - 20)/180*np.pi)**2

##In particluar, that LA_optimality times av. exist velocity correlated well with 
##HR%. So, let's add that too



standard = pd.read_csv('../data/hitters_since_1947.csv')
standard = standard[standard['Season'] >= 2016]
standard.drop(['G','AB','AVG','R','RBI'], axis = 1, inplace=True)

statcast_cols = statcast.select_dtypes(exclude = 'object').drop(['playerid','Season'], axis = 1).columns 
standard_cols = standard.select_dtypes(exclude = 'object').drop(['playerid','Season'], axis = 1).columns

df = pd.merge(statcast, standard.select_dtypes(exclude='object'), on = ['playerid','Season'], how = 'left').dropna()

In [168]:
df.rename({'PA_y':'PA'}, axis=1, inplace = True)
for s in standard_cols:
    df[s+'%'] = df[s]/df['Events']
df.columns


Index(['Season', 'Name', 'Team', 'PA_x', 'Events', 'EV', 'maxEV', 'LA',
       'Barrels', 'Barrel%', 'HardHit', 'HardHit%', 'playerid', 'cosLA',
       'sinLA', 'EV_x', 'EV_y', 'Hangtime', 'Distance', 'LA_optimality', 'PA',
       'H', '1B', '2B', '3B', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH',
       'GDP', 'SB', 'CS', 'PA%', 'H%', '1B%', '2B%', '3B%', 'HR%', 'BB%',
       'IBB%', 'SO%', 'HBP%', 'SF%', 'SH%', 'GDP%', 'SB%', 'CS%'],
      dtype='object')

In [169]:
def multiseason_lines(df, stats, num_seasons):
    """
    Adds columns to rows in a DataFrame for previous seasons; useful for making past seasons features
    in ML applicaitons.

    Parameters
    ----------
    df: DataFrame
        A DataFrame that includes multiple seasons of player data in separate rows.
        The DataFrame must include a unique 'playerid' column (per Fangraphs) and 
        the seasons must be labeled 'Season' with int-type data.
    stats: list-like
        The stats to include from previous seasons.

    num_seasons: int
        The number of past seasons to include as past stats.

    Returns
    -------
        A DataFrame with the columns from df and additional columns from stats
        labeled with suffixes '_1', '_2'...'_x' for the past season's stats.

    Example
    -------
        >>> df
        playerid | Season | Player | Batting Average
        001        2001     Ichiro   .350
        001        2002     Ichiro   .321
        >>> multiseason_lines(df, ['Batting_Average'], 1)
        playerid | Season | Player | Batting Average | Batting Average_1
        001        2002     Ichiro   .350              .321
    """
    out = df.copy()
    for n in range(num_seasons):
        df1 = df.copy()
        df1['Season'] = df1['Season'] + 1 + n ##we set df1 to match df, except that all season numbers are incremented.
        out = pd.merge(out, df1[stats], how = 'left', on=['playerid','Season'], #then we can align a previous season with a current one.
            suffixes=("","_"+str(1+n)))
    return out



In [170]:
df['Events']

0       3
1       1
2       1
3       2
4       2
       ..
4034    1
4035    1
4036    1
4037    1
4038    1
Name: Events, Length: 3561, dtype: int64

In [171]:
stats = df.select_dtypes(exclude='object').columns
df = multiseason_lines(df,stats,1).dropna()
event_threshold = 100
df = df[(df['Events'] > event_threshold) & (df['Events_1']> event_threshold)]
df.columns

Index(['Season', 'Name', 'Team', 'PA_x', 'Events', 'EV', 'maxEV', 'LA',
       'Barrels', 'Barrel%', 'HardHit', 'HardHit%', 'playerid', 'cosLA',
       'sinLA', 'EV_x', 'EV_y', 'Hangtime', 'Distance', 'LA_optimality', 'PA',
       'H', '1B', '2B', '3B', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH',
       'GDP', 'SB', 'CS', 'PA%', 'H%', '1B%', '2B%', '3B%', 'HR%', 'BB%',
       'IBB%', 'SO%', 'HBP%', 'SF%', 'SH%', 'GDP%', 'SB%', 'CS%', 'PA_x_1',
       'Events_1', 'EV_1', 'maxEV_1', 'LA_1', 'Barrels_1', 'Barrel%_1',
       'HardHit_1', 'HardHit%_1', 'cosLA_1', 'sinLA_1', 'EV_x_1', 'EV_y_1',
       'Hangtime_1', 'Distance_1', 'LA_optimality_1', 'PA_1', 'H_1', '1B_1',
       '2B_1', '3B_1', 'HR_1', 'BB_1', 'IBB_1', 'SO_1', 'HBP_1', 'SF_1',
       'SH_1', 'GDP_1', 'SB_1', 'CS_1', 'PA%_1', 'H%_1', '1B%_1', '2B%_1',
       '3B%_1', 'HR%_1', 'BB%_1', 'IBB%_1', 'SO%_1', 'HBP%_1', 'SF%_1',
       'SH%_1', 'GDP%_1', 'SB%_1', 'CS%_1'],
      dtype='object')

In [172]:
correlations = df.corr()
df['PA_x'].min()

144

In [173]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

target = df['HR%']
data = df[[ x for x in df.select_dtypes(exclude='object').columns if x.endswith("_1")]]
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [174]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for e in data.columns:
    x = scaler.fit_transform(target.to_frame())
    y = scaler.fit_transform(data[e].to_frame())
    print("{}: {:.2f}".format(e, r2_score(x,y)))

PA_x_1: -0.69
Events_1: -0.97
EV_1: 0.09
maxEV_1: 0.08
LA_1: -0.27
Barrels_1: 0.11
Barrel%_1: 0.35
HardHit_1: -0.44
HardHit%_1: 0.16
cosLA_1: -1.73
sinLA_1: -0.27
EV_x_1: -0.46
EV_y_1: -0.20
Hangtime_1: -0.20
Distance_1: -0.13
LA_optimality_1: -0.32
PA_1: -0.69
H_1: -0.84
1B_1: -1.22
2B_1: -0.70
3B_1: -1.24
HR_1: 0.03
BB_1: -0.41
IBB_1: -0.62
SO_1: -0.15
HBP_1: -0.82
SF_1: -0.84
SH_1: -1.66
GDP_1: -0.89
SB_1: -1.34
CS_1: -1.35
PA%_1: -0.07
H%_1: -0.45
1B%_1: -1.60
2B%_1: -0.49
3B%_1: -1.31
HR%_1: 0.26
BB%_1: -0.30
IBB%_1: -0.61
SO%_1: -0.13
HBP%_1: -0.87
SF%_1: -0.85
SH%_1: -1.58
GDP%_1: -0.89
SB%_1: -1.38
CS%_1: -1.37


# Establishing a Baseline

Any decent information we get should improve on using a simple linear regression of a previous season's HR total.
So, we findout how that scores.

...

They scored roughly .37 and .40; the test was actually a little higher than the train, which is just a result of the split (random chance.)


In [155]:
linear = LinearRegression().fit(X_train['HR%_1'].to_frame(), y_train)

print(linear.score(X_train['HR%_1'].to_frame(), y_train),
    linear.score(X_test['HR%_1'].to_frame(), y_test)) ##rough result: .35 and .39 (!)

0.4116271330202401 0.3303701838274501


In [175]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha = .001).fit(X_train,y_train)
#cols = ['maxEV_1', 'Barrels_1', 'HardHit_1','HardHit%_1']
linear.fit(X_train,y_train)
print(linear.score(X_train, y_train),
    linear.score(X_test, y_test))
print(lasso.score(X_train, y_train),
    lasso.score(X_test,y_test))

0.5610194416601278 0.4289615694265221
0.5334230008827752 0.464247907011741


In [157]:
print(lasso.intercept_)
for e in list(zip(X_train.columns,lasso.coef_)):
    if e[1] != 0:
        print('{:10}:  {:.6f}'.format(e[0],e[1]*100))


0.039703346437611926
1B_1      :  -0.033322
2B_1      :  0.015486
3B_1      :  -0.008434
HR_1      :  0.114921
BB_1      :  0.004539
SO_1      :  0.014939
HBP_1     :  -0.010778
SH_1      :  -0.120601
GDP_1     :  -0.015971
SB_1      :  -0.011189
CS_1      :  -0.007599


In [158]:
##a simple function for checking out a model

def quick_check(model, data, target, random_seed=1234):
    X_train, X_test, y_train, y_test = train_test_split(data, target, random_state = random_seed)
    model.fit(X_train,y_train)
    print("Model train score:\n   {}".format(model.score(X_train,y_train)))
    print("Model test score:\n   {}".format(model.score(X_test,y_test)))
    print()

## Hmmm...

The above cell show some of the usual linear regression perversion: R_1 and RBI_1 have non-zero coefs while HR_1 is 0!

Let's try a minimal set of features.

In [176]:
#quick_check(lasso,data,target)
#print(data.columns)
#print(statcast_cols)
#print(standard_cols)
#print(data.columns)
x = [x+'_1' for x in standard_cols if x not in ['PA','Events']]
x = x + [x+'%_1' for x in standard_cols if x not in ['PA','Events']]
x = data[x]
y = [x+'_1' for x in statcast_cols if x not in ['PA','Events']]
y = data[y]
#quick_check(linear, x, target)
#print(x.head())
quick_check(lasso, data, target)
quick_check(lasso, x, target)
quick_check(lasso, y, target)

Model train score:
   0.5307408461268874
Model test score:
   0.49399582510083195

Model train score:
   0.423557079333597
Model test score:
   0.3757625222523765

Model train score:
   0.5193259868530453
Model test score:
   0.4916715234538037



In [182]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from random import randint

lasso_pipe = Pipeline(
    steps=[('scaler',scaler),
        #('poly',PolynomialFeatures(degree=2)),
        ('lasso',lasso)
    ]

)

lasso.alpha = .001
quick_check(lasso_pipe, data, target)
quick_check(lasso_pipe, x, target)
quick_check(lasso_pipe, y, target)

data1 = y
z = 0
times = 500
for i in range(times):
    rand = randint(1,100_000)
    X_train, X_test, y_train, y_test = train_test_split(data1,target)
    lasso_pipe.fit(X_train,y_train)
    z += lasso_pipe.score(X_test, y_test)
print(z/times)

lasso.fit(scaler.fit_transform(y),target)
print('\n'+str(lasso.intercept_))
list(zip(y.columns, lasso.coef_))
    

Model train score:
   0.5255423014372425
Model test score:
   0.486244126242353

Model train score:
   0.45610687258592164
Model test score:
   0.40571036378810654

Model train score:
   0.5113419993721178
Model test score:
   0.4780409837273507

0.49577206537676527

0.049924336145080975


[('EV_1', 0.0),
 ('maxEV_1', 0.004899862445949801),
 ('LA_1', 0.0),
 ('Barrels_1', -0.0),
 ('Barrel%_1', 0.009619400997504789),
 ('HardHit_1', -0.00012384252362684936),
 ('HardHit%_1', 0.002386144745849478),
 ('cosLA_1', -0.0),
 ('sinLA_1', 0.0),
 ('EV_x_1', 0.0),
 ('EV_y_1', 0.0),
 ('Hangtime_1', 0.0),
 ('Distance_1', 0.004674760649049382),
 ('LA_optimality_1', 0.0)]

# Refelcting on results so far   

Here are the results of the above cell, run with data1 = x, y, data:

```
data 0.49717945803508745

y 0.49779010748919456

x 0.42453997365151097
```

What does this imply? For the available data, statcast data is a better predictor of future homeruns than past standard baseball data. The union of those two data sets doesn't outperform statcast alone.

This suggests that there's a lot packed into the statcast data.

I'm curious about whether a different model could get a higher score. (Earlier testing suggests that Lasso is very strong for this process.)


In [161]:
from sklearn.ensemble import GradientBoostingRegressor

boosting = GradientBoostingRegressor(n_estimators=100)

quick_check(boosting, data, target)
quick_check(boosting, y, target)
##major overfit

boosting.n_estimators = 15
quick_check(boosting, data, target)
quick_check(boosting, y, target)


Model train score:
   0.8192867097521981
Model test score:
   0.4697349737781925

Model train score:
   0.7492420454335325
Model test score:
   0.4289051515414324

Model train score:
   0.5708625015227997
Model test score:
   0.41917436654198914

Model train score:
   0.5468818111980864
Model test score:
   0.4105303429713306



In [162]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
#MLPRegressor? There is almost certainly too little data. 

mlp = MLPRegressor(solver = 'lbfgs',hidden_layer_sizes=[4])

layers = {'hidden_layer_sizes': [4,10,15,20,30]}
alpha = {'alpha': [.0001,.001,.01,.1,1]}
active = {'activation': ['identity', 'logistic', 'tanh', 'relu']}

grid = GridSearchCV(mlp, param_grid=active)

grid.fit(y,target)


GridSearchCV(estimator=MLPRegressor(hidden_layer_sizes=[4], solver='lbfgs'),
             param_grid={'activation': ['identity', 'logistic', 'tanh',
                                        'relu']})

In [163]:
grid.cv_results_

{'mean_fit_time': array([0.12659855, 0.01640606, 0.01120844, 0.05272746]),
 'std_fit_time': array([0.05765833, 0.00596163, 0.00419245, 0.03874153]),
 'mean_score_time': array([0.00272145, 0.00263338, 0.00321093, 0.00286779]),
 'std_score_time': array([4.04260057e-05, 4.67736663e-05, 5.12629366e-04, 3.10828071e-04]),
 'param_activation': masked_array(data=['identity', 'logistic', 'tanh', 'relu'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'activation': 'identity'},
  {'activation': 'logistic'},
  {'activation': 'tanh'},
  {'activation': 'relu'}],
 'split0_test_score': array([ 0.01514067, -0.90709553, -0.91081548, -0.07590882]),
 'split1_test_score': array([-1.61544851, -0.35026086, -0.40750631,  0.10121933]),
 'split2_test_score': array([ 0.16315358, -0.00115373, -0.001028  , -0.00501897]),
 'split3_test_score': array([ 0.27187935, -0.37182163, -0.35151315, -0.35332221]),
 'split4_test_score': array([ -0.29948404,  -

## Feature Check

We know that an optimum launch angle for distance is more like 30 degrees than 45 degrees. Drag reduces velocity and that means that a ball hit at 45 degrees tends to stall.

Let's add a feature that's maximized when launch angle is 30 degrees.

In [164]:
la_opt = df[['Name','Season','Events','playerid','EV','LA','HR%']]
angles = [16+x for x in [1,2,3,4,5,6]]
for a in angles:
    opt = np.cos((la_opt['LA'] - a)/180*np.pi)**2
    la_opt['ang_'+str(a)] = opt
    la_opt['ang*EV_'+str(a)] = opt*la_opt['EV']

In [165]:
la_opt.corr()

,Season,Events,playerid,EV,LA,HR%,ang_17,ang*EV_17,ang_18,ang*EV_18,ang_19,ang*EV_19,ang_20,ang*EV_20,ang_21,ang*EV_21,ang_22,ang*EV_22
Season,1.000000,-0.069817,0.223594,0.257816,0.085847,0.108926,0.070521,0.252030,0.074487,0.248569,0.077121,0.244174,0.078945,0.239160,0.080255,0.233782,0.081225,0.228236
Events,-0.069817,1.000000,0.053294,0.154813,0.002914,0.087148,0.033103,0.146987,0.028985,0.141037,0.025720,0.134878,0.023099,0.128692,0.020962,0.122615,0.019190,0.116739
playerid,0.223594,0.053294,1.000000,0.095406,0.057803,0.108253,0.083048,0.121707,0.080651,0.121614,0.078433,0.120924,0.076471,0.119788,0.074760,0.118338,0.073270,0.116677
EV,0.257816,0.154813,0.095406,1.000000,0.081934,0.612653,0.069731,0.881222,0.073168,0.850751,0.075409,0.818617,0.076930,0.785918,0.078002,0.753474,0.078782,0.721858
LA,0.085847,0.002914,0.057803,0.081934,1.000000,0.429745,0.847392,0.471043,0.890094,0.535728,0.918020,0.592458,0.937053,0.641832,0.950509,0.684598,0.960325,0.721549
HR%,0.108926,0.087148,0.108253,0.612653,0.429745,1.000000,0.349038,0.686294,0.369581,0.693869,0.383308,0.697422,0.392866,0.697810,0.399768,0.695793,0.404912,0.692004
ang_17,0.070521,0.033103,0.083048,0.069731,0.847392,0.349038,1.000000,0.532836,0.996261,0.581411,0.988466,0.623056,0.979451,0.658455,0.970413,0.688371,0.961833,0.713562
ang*EV_17,0.252030,0.146987,0.121707,0.881222,0.471043,0.686294,0.532836,1.000000,0.534003,0.997194,0.532229,0.989690,0.529263,0.978746,0.525901,0.965418,0.522507,0.950551
ang_18,0.074487,0.028985,0.080651,0.073168,0.890094,0.369581,0.996261,0.534003,1.000000,0.586225,0.997854,0.631204,0.993214,0.669625,0.987645,0.702264,0.981879,0.729900
ang*EV_18,0.248569,0.141037,0.121614,0.850751,0.535728,0.693869,0.581411,0.997194,0.586225,1.000000,0.586954,0.997636,0.585777,0.991353,0.583739,0.982227,0.581354,0.971134


In [200]:
y['LA*EV_1'] = y['LA_optimality_1']*y['EV_1']
X_train,X_test,y_train,y_test = train_test_split(y,target)
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test)

0.5183386220544914

In [197]:
y.shape

(893, 15)